In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix, ConfusionMatrixDisplay,r2_score

In [2]:
df = pd.read_csv(r"C:\Users\Aniket\OneDrive\Desktop\Personal\Projects\IPO-Performance-Prediction\Notebooks\IPO.csv").copy()
df_ = pd.read_csv(r"C:\Users\Aniket\OneDrive\Desktop\Personal\Projects\IPO-Performance-Prediction\Notebooks\Validate_IPO.csv").copy()

In [3]:
df['listing date'] = pd.to_datetime(df['listing date'])

In [4]:
df.head().T

,0,1,2,3,4
company,A G Universal,AA Plus Tradelink,AAA Technologies,AB Cotspin,AB Infrabuild
ltp,60.0,0.9,89.1,409.0,179.4
market cap(in cr),32.9,21.4,114.3,671.5,1146.3
listing open,60.0,16.6,44.5,38.5,29.0
listing close,60.1,17.5,43.6,40.4,29.1
issue type,SME,SME,SME,SME,SME
listing date,2023-04-24 00:00:00,2021-07-22 00:00:00,2020-10-13 00:00:00,2022-01-11 00:00:00,2019-07-11 00:00:00
issue size (in Cr),8.7,6.5,10.2,10.1,12.8
issue price (in rupee),60.0,18.0,42.0,35.0,29.0
qib subscription,38.9,38.9,38.9,38.9,38.9


In [5]:
df.shape

(505, 32)

In [9]:
df.columns

Index(['company', 'ltp', 'market cap(in cr)', 'listing open', 'listing close',
       'issue type', 'listing date', 'issue size (in Cr)',
       'issue price (in rupee)', 'qib subscription', 'hni subscription',
       'retail subscription', 'total subscription', 'listing gain %',
       'current gain %', 'lot size', 'sector', 'industry',
       'minimum_investment_rupee', 'subscription_status', 'min_price_range',
       'max_price_range', 'no_of_share', 'change_in_cffoa', 'change_in_ebitda',
       'change_in_eps', 'change_in_net_profit', 'change_in_npm_percent',
       'change_in_roce_percent', 'change_in_roe_percent',
       'change_in_debt_to_equity_ratio', 'CAGR'],
      dtype='object')

In [6]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
ltp,505.0,408.678218,0.9,50.0,143.6,415.0,6753.0,723.221653
market cap(in cr),505.0,5680.266931,3.3,56.0,245.1,1918.4,564126.5,28640.957115
listing open,505.0,240.687129,6.4,51.0,101.0,251.2,2607.5,355.830522
listing close,505.0,241.134455,1.2,51.5,103.4,257.9,3118.7,357.62582
listing date,505,2022-05-02 02:25:25.544554496,2019-02-06 00:00:00,2021-08-10 00:00:00,2022-09-15 00:00:00,2023-07-13 00:00:00,2023-12-29 00:00:00,NaN
issue size (in Cr),505.0,425.838416,1.3,9.3,24.4,387.9,21000.0,1502.698797
issue price (in rupee),505.0,193.417842,10.0,45.0,80.0,195.0,2150.0,276.565922
qib subscription,505.0,44.112475,0.5,32.4,38.9,38.9,228.9,39.742087
hni subscription,505.0,86.419802,0.1,1.9,7.4,87.6,1044.5,165.993369
retail subscription,505.0,53.393069,0.0,1.9,8.7,39.9,1023.1,125.818553


In [7]:
#### Creating Classification feature for prediction

In [8]:
def convert(row):
    if row['CAGR'] > 15:
        return 2
    elif row['CAGR'] > 5 and row['CAGR'] < 15:
        return 1
    else:
        return 0

In [9]:
def convert_regre_class(df):
    return(
        df
        .assign(
            performance = lambda df_ : df_.apply(convert,axis=1)
        )
        # .drop(['current gain %','company','listing date','ltp'],axis=1)
    )

In [10]:
ipo = convert_regre_class(df)
ipo_ = convert_regre_class(df_)

In [11]:
# ipos = convert_regre_class(df)

In [12]:
# ipos = ipos.drop(columns=['company','sector','industry','listing date'])

In [13]:
# ipos.loc[ipos['issue type'] == 'SME',['issue type']] = 0
# ipos.loc[ipos['issue type'] == 'MainBoard',['issue type']] = 1

In [14]:
# ipos['issue type'] = ipos['issue type'].astype('int')

In [15]:
# ipos['subscription_status'].unique()

In [16]:
# ipos.loc[ipos['subscription_status'] == 'Oversubscribed'] = 1
# ipos.loc[ipos['subscription_status'] == 'Undersubscribed'] = 0

In [17]:
# ipos['subscription_status'] = ipos['subscription_status'].astype('int')

In [18]:
# ipos.info()

In [19]:
ipo = ipo[columns_to_be_kept]

NameError: name 'columns_to_be_kept' is not defined

In [ ]:
x = ipo.drop(['performance'],axis=1)
y = ipo['performance']

In [ ]:
ipo_valid = convert_regre_class(df_validation)

In [ ]:
columns_to_be_kept

In [ ]:
ipo_valid = ipo_valid[columns_to_be_kept]
ipo_valid.head(1)

In [ ]:
ipo_valid.loc[ipo_valid['issue type'] == 'SME',['issue type']] = 0
ipo_valid.loc[ipo_valid['issue type'] == 'MainBoard',['issue type']] = 1

In [ ]:
ipo_valid['issue type'] = ipo_valid['issue type'].astype('int')

In [ ]:
x_ipo_valid = ipo_valid.drop('performance',axis=1)

In [ ]:
y_ipo_valid = ipo_valid['performance']

In [ ]:
x_ipo_valid.head(1)

In [ ]:
sample = np.array([428.0,527.3,27.7,119.0,1974.4,90.0,1600,128000 ,33.0,-68.75,66.25,-36.7674,-19.4508,-100,0]).reshape(1,-1)

In [ ]:
sample

In [ ]:
columns_to_be_kept = []

In [ ]:
columns_to_be_kept.extend(['market cap(in cr)',
 'issue size (in Cr)',
 'hni subscription',
 'retail subscription',
 'total subscription',
 'listing gain %',
 'change_in_cffoa',
 'change_in_ebitda',
 'change_in_net_profit',
 'change_in_roce_percent',
 'change_in_roe_percent',
 'change_in_debt_to_equity_ratio',
 'performance'])

In [ ]:
ipo = ipo[columns_to_be_kept]
ipo_ = ipo_[columns_to_be_kept]

In [ ]:
ipo.head(1)

In [ ]:
ipo_.head(1)

In [ ]:
ipo['performance'].value_counts()

In [ ]:
validation = ipo_.drop('performance',axis=1)
actual = ipo_['performance']

##### Train test split

In [ ]:
x = ipo.drop(['performance'],axis=1)
y = ipo['performance']

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=42,test_size=0.2)

##### Logistic Regression

In [ ]:
lg = LogisticRegression(class_weight="balanced", max_iter=500)

In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [ ]:
lg.fit(x_train_scaled,y_train)

In [ ]:
y_pred = lg.predict(x_test_scaled)
y_pred_on_train = lg.predict(x_train_scaled)
y_pred_val = lg.predict(validation)

In [ ]:
print("Train Accuracy:", accuracy_score(y_train, y_pred_on_train))
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Validation Accuracy (2024):", accuracy_score(actual, y_pred_val))

In [ ]:
print("Train r2 score:", r2_score(y_train, y_pred_on_train))
print("Test r2 score:", r2_score(y_test, y_pred))
print("Validation r2 score (2024):", r2_score(actual, y_pred_val))

In [ ]:
print(f"Classification Report of Logistic on Train :\n{classification_report(y_pred_on_train,y_train)}")
print("="*80)
print(f"Classification Report of Logistic on Test :\n{classification_report(y_pred,y_test)}")
print("="*80)
print(f"Classification Report of Logistic on Validation :\n{classification_report(y_pred_val,actual)}")

##### Random Forest

In [ ]:
rfc = RandomForestClassifier(n_estimators=200, class_weight="balanced", n_jobs=-1)

In [ ]:
rfc.fit(x_train,y_train)

In [ ]:
y_pred = rfc.predict(x_test)
y_pred_on_x = rfc.predict(x_train)
y_pred_val = rfc.predict(validation)

In [ ]:
print("Train Accuracy:", accuracy_score(y_train, y_pred_on_x))
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Validation Accuracy (2024):", accuracy_score(actual, y_pred_val))

In [ ]:
print("Train r2 score:", r2_score(y_train, y_pred_on_x))
print("Test r2 score:", r2_score(y_test, y_pred))
print("Validation r2 score (2024):", r2_score(actual, y_pred_val))

In [ ]:
print(f"Classification Report of Random Forest on Train :\n{classification_report(y_pred_on_x,y_train)}")
print("="*80)
print(f"Classification Report of Random Forest on Test :\n{classification_report(y_pred,y_test)}")
print("="*80)
print(f"Classification Report of Random Forest on Validation :\n{classification_report(y_pred_val,actual)}")

### Xgboost

In [ ]:
##### Withpout class imbalancing

In [ ]:
xgb_classifier = xgb.XGBClassifier(objective='multi:softmax',  num_class=4)

In [ ]:
xgb_classifier.fit(x_train,y_train)

In [ ]:
y_pred = xgb_classifier.predict(x_test)
y_pred_on_x = xgb_classifier.predict(x_train)
y_pred_val = xgb_classifier.predict(validation)

In [ ]:
print("Train Accuracy:", accuracy_score(y_train, y_pred_on_x))
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Validation Accuracy (2024):", accuracy_score(actual, y_pred_val))

In [ ]:
print(f"Classification Report of Random Forest on Train :\n{classification_report(y_pred_on_x,y_train)}")
print("="*80)
print(f"Classification Report of Random Forest on Test :\n{classification_report(y_pred,y_test)}")
print("="*80)
print(f"Classification Report of Random Forest on Validation :\n{classification_report(y_pred_val,actual)}")

#### Applying regularization

In [ ]:
xgb_c = xgb.XGBClassifier(
    objective="multi:softmax",
    num_class=5,
    eval_metric="mlogloss",
    max_depth=3,        # lower tree depth
    min_child_weight=5, # require more samples per leaf
    subsample=0.7,      # random row sampling
    colsample_bytree=0.6, # random feature sampling
    reg_lambda=0.9,       # L2 regularization
    reg_alpha=2.8        # L1 regularization
)

L2 = 0.9
L1 = 2.8

In [ ]:
xgb_c.fit(x,y)

In [ ]:
y_pred = xgb_c.predict(x_test)
y_pred_on_x = xgb_c.predict(x_train)
y_pred_val = xgb_c.predict(validation)

In [ ]:
print("Train Accuracy:", accuracy_score(y_train, y_pred_on_x))
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Validation Accuracy (2024):", accuracy_score(actual, y_pred_val))

In [ ]:
print(f"Classification Report of Random Forest on Train :\n{classification_report(y_pred_on_x,y_train)}")
print("="*80)
print(f"Classification Report of Random Forest on Test :\n{classification_report(y_pred,y_test)}")
print("="*80)
print(f"Classification Report of Random Forest on Validation :\n{classification_report(y_pred_val,actual)}")

In [ ]:
print("Train r2 score:", r2_score(y_train, y_pred_on_x))
print("Test r2 score:", r2_score(y_test, y_pred))
print("Validation r2 score (2024):", r2_score(actual, y_pred_val))

In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [ ]:
svm_model = SVC(kernel='rbf', C=1.0) # rbf kernel is a good starting point
svm_model.fit(x_train_scaled, y_train)

In [ ]:
y_pred = svm_model.predict(x_test_scaled)

In [ ]:
accuracy_score(y_pred,y_test)

In [ ]:
print(classification_report(y_test, y_pred, zero_division=0))

In [ ]:
def plot_numeric_graph(col):

    fig,ax = plt.subplots(2,2,figsize=(15,10))

    ax[0,0].hist(df[col],density=True)
    # sns.histplot(data=df,x=col,ax=ax[0,0],bins=np.histogram_bin_edges(df[col]),binwidth=10)
    ax[0,0].set_title(f'Histogram & KDE of {col}')
    ax[0,1].plot(df[col])
    ax[0,1].set_title(f'Line Chart of {col}')
    sns.kdeplot(ax=ax[0,0],data=df,x=col)
    sns.boxplot(data=df,x=col,ax=ax[1,0])
    ax[1,0].set_title(f'Box plot of {col}')
    plt.show()

In [ ]:
for i in numeric_columns:
    plot_numeric_graph(i)
    print("="*140)
    print("\n")
    # break

**All notes**
- `LTP`
  1. LTP has major values lie between 0-~500.
  2. As it crosses ~500 price ltp start declining.
  3. There is a `major drop` in LTP after 500 and after 500 there is not drop in LTP it is decling.
- `Market Cap`
  1. Some companies have very large market cap 500000 cr.
  2. While major companies have mrket cap in between 0-~50000 cr.
  3. In our data majorly we have companies whose market cap lies between 0-50000 cr.
- `Listing Open`
  1. Majorly IPO's open price range approximately 0-200.
  2. After 250 rs. approx we can see a clear and major drop in opening price. And as price increases the no. of companies decreases.
  3. We have very fewer companies who have higher opening price .
- `Closing Price`
  1. Same pattern as in opening price.
- `Issue Size`
  1. We can se a very similar pattern between market cap and issue size . Not numerically similar but the pattern both are making is similar.
  2. Majorly companies issue their share size in  betwwen ~ 0-2500 cr.
  3. Same here as we cross approximately 2500 cr there is sudden drop in issue price. And then drop is conitnouos. Means majorly we have only that companies whose issue size is in between 0-2500 cr approximately .
- `Issue Price`
  1. Issue price also show some type of association with opening price.
  2. We have more issue price in between 0-250 rs. approx and as the issue price increase the no. of companies decrease.
- `QIB Subscription`
  1. Same here we can see major times QIB gets sucbscribed in range 0f 0-25 approx and as we cross the there is sudden drop in subscription.
  2. And the drop continuous but not a rapid rate.
  3. We can see a kde plot start from nearly -40 and make a center at around 25.
- `HNI Subscription`
  1. HNI also get more subscribed in between 0-100.
  2. And after 100 we can notice a big drop and the drop is continuous after some number and it is sudden but not a bigger. BUt when we see it in kde it is smooother.
  3. Here also Kde plot start from nearly -170 and make a center nearly 0.
- `Retail Subscription`
  1. Same pattern as both subscription shows.
  2. And also it's kde start from negative and amke a center around at 0.
- `Total Subscription`
  1. It is also very similar to QIB,HNI and retail subscription.
  2. Majorly total subscription is lies between 0-75 approx and the a bigger drop and this continues.
  3. The drop happen after 1st and bigger drop but not bigger as first.
  4. And also kde for this is also start from negative and make a center around 0.
- `Listing Gain`
  1. We can se here also the data is right skewed means modt of data we have are at right side at fewer are left side.
  2. Here we can see may companies have negative gain on their listing date around -50 to below 0.
  3. And are there are major companies who have listong gain between 0-50. 
  4. And also there are companies who lies in between above 50 to 100 % gain.
  5. And above percent also. Means there are companies who achieve 100 % gain on their listing day.
- `Current Gain`
  1. But here we can see a different story.
  2. Although the kde start from negative but majorly companies have above 100 or 500 % gain in market.
  3. This is also a right skewed data.
- `Lot Size`
  1. We can Lot size differ vary.
  2. Though this also follows right skewed distribution.
  3. we have a peak at different-different lot size range .
  4. Such as a peak at between 5000-6000 and then at 7000-8000 and then 9000-10000.
  5. And also greater peak at 0-nearly 1000 lot. And then a drop after 1000 lot size and similarly drop  after 3000 lot size.
  6. And we can also we don't have any lot size between 4000-500 ,6000-7000,8000-9000 and then after 10000.
- `Minimum Investment Rupee`
  1. Here data have multiple peak at different values.
  2. Mojorly values lies between 1L-1.5L
- `Min and Max Price Range`
  1. Both are showing same pattern.
  2. We can see in min price range their is gap between from roughly 12,000 - 15,000.
  3. Min price have some peak at price range of 15,000 but in max price range this faltter than min in min price range.
- `No. of Shares`
  1. Most of IPO's launch 0-~700 lakh shares. Becuase at this region their is a peak.
  2. In data their are companies who launch their IPO with more than 2000 lakh shares.

In [ ]:
def plot_cat_graph(col):
    fig,ax = plt.subplots(2,2,figsize=(15,10))

    sns.barplot(data=df, x=col ,y=df['market cap(in cr)'], ax = ax[0,0])
    # sns.swarmplot(data=df, x=col ,y=df['market cap(in cr)'], ax= ax[0,1])
    sns.violinplot(x=col, y=df['market cap(in cr)'] , data=df,ax=ax[1,0])
    sns.stripplot(x=col, y=df['market cap(in cr)'] , data=df, jitter=True,ax=ax[1,1])

In [ ]:
def plot_better_graph(col):
    plt.figure(figsize=(10, 7))

    # 1. Draw the violin plot to show the distribution
    sns.violinplot(data=df, x=col, y='market cap(in cr)', color='lightgray')

    # 2. Overlay a stripplot to show individual points
    sns.stripplot(data=df, x=col, y='market cap(in cr)', jitter=True)

    plt.title(f'Distribution of Market Cap by {col}')
    plt.show()

In [ ]:
for i in cat_columns:
    if i == 'company':
        continue
    plot_better_graph(i)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
ipo = df.copy()

In [ ]:
numeric_columns = ipo.select_dtypes(include='number')
num_col = numeric_columns.columns

In [ ]:
columns_to_be_kept = []

In [ ]:
total_columns = numeric_columns.shape[1]
column_index = 0

for i in range (0,total_columns):
    
    vif_value = variance_inflation_factor(numeric_columns, column_index)
    print (numeric_columns.columns[column_index],' ---> ',vif_value)
    if vif_value <= 6:
        columns_to_be_kept.append(num_col[i])
        column_index = column_index+1
    
    else:
        vif_data = numeric_columns.drop([num_col[i]] , axis=1)
        column_index = column_index+1


In [ ]:
columns_to_be_kept.extend(['market cap(in cr)',
 'issue size (in Cr)',
 'hni subscription',
 'retail subscription',
 'total subscription',
 'listing gain %',
 'change_in_cffoa',
 'change_in_ebitda',
 'change_in_eps',
 'change_in_net_profit',
 'change_in_npm_percent',
 'change_in_roce_percent',
 'change_in_roe_percent',
 'change_in_debt_to_equity_ratio',
 'CAGR',
 'performance'])

In [ ]:
columns_to_be_kept

In [ ]:
ipo_ = ipo[columns_to_be_kept]

In [ ]:
ipo_ = ipo_.drop(['performance'],axis=1)

In [ ]:
x = ipo_.drop('CAGR',axis=1)
y = ipo_['CAGR']

In [ ]:
x_test,x_train,y_test,y_train = train_test_split(x,y,random_state=42,test_size=0.2)

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(x_train,y_train)

In [ ]:
y_pred = lr.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score

In [ ]:
# train_mse = mean_squared_error(y_train, y_pred)
test_mse = mean_squared_error(y_test, y_pred)
# train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred)

In [ ]:
print("Model Performance:")
# print(f"Training MSE: {train_mse:.4f}, R²: {train_r2:.4f}")
print(f"Test MSE: {test_mse:.4f}, R²: {test_r2:.4f}")

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [ ]:
x_test_scaled

In [ ]:
lr.fit(x_train_scaled,y_train)

In [ ]:
y_pred = lr.predict(x_test_scaled)

In [ ]:
mean_squared_error(y_test, y_pred)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfr = RandomForestRegressor(n_estimators=200)

In [ ]:
rfr.fit(x_train,y_train)

In [ ]:
y_pred = rfr.predict(x_test)

In [ ]:
# train_mse = mean_squared_error(y_train, y_pred)
test_mse = mean_squared_error(y_test, y_pred)
# train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred)

In [ ]:
print("Model Performance:")
# print(f"Training MSE: {train_mse:.4f}, R²: {train_r2:.4f}")
print(f"Test MSE: {test_mse:.4f}, R²: {test_r2:.4f}")

In [ ]:
xgb_r = xgb.XGBRegressor(n_estimator=200,reg_lambda=0.8,reg_alpha=0.7,learning_rate=0.2,enable_global_explain=True)

In [ ]:
xgb_r.fit(x_train,y_train)

In [ ]:
y_pred_on_x = xgb_r.predict(x_train)
y_pred = xgb_r.predict(x_test)

In [ ]:
train_mse = mean_squared_error(y_train, y_pred_on_x)
test_mse = mean_squared_error(y_test, y_pred)
train_r2 = r2_score(y_train, y_pred_on_x)
test_r2 = r2_score(y_test, y_pred)

In [ ]:
print("Model Performance:")
print(f"Training MSE: {train_mse:.4f}, R²: {train_r2:.4f}")
print(f"Test MSE: {test_mse:.4f}, R²: {test_r2:.4f}")

In [ ]:
# Get unique class labels
labels = sorted(np.unique(y_test))

# Create the confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=labels)

# Plot the confusion matrix using ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(cmap=plt.cm.Blues, values_format='d')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Define the filename for the model
xg_boost = 'xgb_c.pkl'

# Use pickle to dump the trained model object into a file
with open(xg_boost, 'wb') as file:
    pickle.dump(xgb_c, file)

print(f"Model successfully saved as '{xg_boost}'")

np.int64(1)